In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [3]:
import pandas as pd
import numpy as np
import datetime
!pip install pandas_summary
from pandas_summary import DataFrameSummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 44.1 MB/s 


In [4]:
df = pd.read_feather('/content/drive/MyDrive/Redes Neuronales/TP2/kaggle-rossman-master/train_normalized_data.fth')
df_test = pd.read_feather('/content/drive/MyDrive/Redes Neuronales/TP2/kaggle-rossman-master/test_normalized_data.fth')

In [5]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [6]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [7]:
from lightgbm import LGBMRegressor

In [8]:
y_out_columns = ['Sales']

In [9]:
# split_type = 'random'
# split_type = 'no_split'
split_type = 'last_week'

In [10]:
# Esta es para entrenar con todo
if split_type == 'no_split':
    df_train = df
elif split_type == 'last_week':
    # Esto divide en train y val
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
    df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
elif split_type == 'random':
    # Splitting aleatorio
    np.random.seed(42)
    indexes = np.arange(len(df))
    np.random.shuffle(indexes)
    N = len(df)//5
    df_train = df[N:]
    df_val = df[:N]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [11]:
X_train = df_train[cat_vars + contin_vars]
if split_type != 'no_split':
    X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [12]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    if split_type != 'no_split':
        y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    if split_type != 'no_split':
        y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [13]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [14]:
def calculate_RMSE(X, y, log_output=True):
    y_preds = np.exp(model.predict(X, verbose=1)*max_log_y)
    return np.sqrt((((y - y_preds)/y)**2).sum()/len(y_preds))

In [15]:
X_train.shape

(814150, 38)

In [16]:
def objective(params):
    params = {
        # 'max_depth': int(params['max_depth']),
        # 'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'learning_rate': '{:.4f}'.format(params['learning_rate'])
    }
    
    clf = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
        **params
    )
    
    score = -cross_val_score(clf, X_train.values, y_train.reshape(-1)).mean()
    return score

In [17]:
space = {
    # 'max_depth': hp.quniform('max_depth', 1, 64, 4),
    # 'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'learning_rate': hp.loguniform('learning_rate', -4, -2)
}

best = fmin(fn=objective,
            space=space,
            verbose=2,
            algo=tpe.suggest,
            max_evals=10)

100%|██████████| 10/10 [3:27:58<00:00, 1247.90s/it, best loss: -0.9056798260207215]


In [18]:
best

{'learning_rate': 0.0848502992806897}

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
parameters = {'learning_rate':[0.001, 0.05, 0.1, 0.5], 'max_depth':[1, 7, 31, 63]}
grid_search_res_1 = {'learning_rate': 0.05, 'min_child_samples': 5, 'max_depth': -1}
grid_search_res_2 = {'learning_rate': 0.05, 'max_depth': 7, 'min_child_samples': 20}

In [21]:
min_child_samples = 5
n_estimators = 4000
learning_rate = 0.1
model = LGBMRegressor(n_estimators=n_estimators, **grid_search_res_2)

In [22]:
clf = GridSearchCV(model, parameters, n_jobs=-1, verbose=1, cv=3)

In [23]:
fit_params={ "eval_metric" : 'l2', 
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars}
if split_type != 'no_split':
    fit_params["eval_set"] = [(X_val, y_val.reshape(-1))]
    fit_params["early_stopping_rounds"] = 100

In [24]:
# clf.fit(X_train, y_train.reshape(-1), **fit_params)
model.fit(X_train, y_train.reshape(-1), **fit_params)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.000272121	valid_0's l2: 0.000272121
[200]	valid_0's l2: 0.00020163	valid_0's l2: 0.00020163
[300]	valid_0's l2: 0.000163379	valid_0's l2: 0.000163379
[400]	valid_0's l2: 0.000148547	valid_0's l2: 0.000148547
[500]	valid_0's l2: 0.000138178	valid_0's l2: 0.000138178
[600]	valid_0's l2: 0.000131548	valid_0's l2: 0.000131548
[700]	valid_0's l2: 0.00012836	valid_0's l2: 0.00012836
[800]	valid_0's l2: 0.000125261	valid_0's l2: 0.000125261
[900]	valid_0's l2: 0.000122757	valid_0's l2: 0.000122757
[1000]	valid_0's l2: 0.000120754	valid_0's l2: 0.000120754
[1100]	valid_0's l2: 0.000119432	valid_0's l2: 0.000119432
[1200]	valid_0's l2: 0.000118822	valid_0's l2: 0.000118822
[1300]	valid_0's l2: 0.000118132	valid_0's l2: 0.000118132
[1400]	valid_0's l2: 0.000117226	valid_0's l2: 0.000117226
[1500]	valid_0's l2: 0.000116814	valid_0's l2: 0.000116814
[1600]	valid_0's l2: 0.000116647	valid_0's l2: 0.000116647
[1700]

LGBMRegressor(learning_rate=0.05, max_depth=7, n_estimators=4000)

In [25]:
clf

GridSearchCV(cv=3,
             estimator=LGBMRegressor(learning_rate=0.05, max_depth=7,
                                     n_estimators=4000),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.05, 0.1, 0.5],
                         'max_depth': [1, 7, 31, 63]},
             verbose=1)

In [26]:
# clf.best_params_

In [27]:
# clf.best_score_

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [28]:
model.score(X_val, y_val)

0.919620999093268

In [29]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [30]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

0.08966946144178467

In [31]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

0.11929344800997081

In [32]:
calculate_RMSE(X_train, df_train['Sales'].values)

0.08966946144178467

In [33]:
calculate_RMSE(X_val, df_val['Sales'].values)

0.11929344800997081

# Sumbit a la competición

In [35]:
sample_csv = pd.read_csv('/content/drive/MyDrive/Redes Neuronales/TP2/rossman/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_lightgbm_{split_type}-{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv', index=False)

In [36]:
from google.colab import files

files.download(f'submision_lightgbm_{split_type}-{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
pip freeze > requirements.txt

In [38]:
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>